# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION={'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
#### sessionId = 338, and itemInSession = 4
#### CQL query would be :
#### SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4
#### We will name our table songs_details
#### Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
#### The columns of our table will be: sessionId, itemInSession, artist, song_title and song_length

In [8]:
create_table1="CREATE TABLE IF NOT EXISTS songs_details"
create_table1=create_table1+("""(sessionId INT,itemInSession INT,artist text,song_title text,song_length FLOAT,PRIMARY KEY(sessionId,itemInSession))""")
try:
    session.execute(create_table1)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_details(sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
#select statment for table 1
try:
    rows = session.execute(""" SELECT artist, song_title, song_length FROM songs_details
                    WHERE sessionId = 338 
                    AND itemInSession = 4  
                    """)

except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist,row.song_title,row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query2:SELECT itemInSession, artist, song, firstName, lastName FROM artist_details WHERE userId = 10 AND sessionId = 182
#### we will name our table artist_details
#### Our primary key will consist of composite partition key userId, sessionId. The reason for this is that if we only use userId as partition key, 
#### sessionid which belongs to the same user will be put into different nodes, which will have the performance issue when the volume of data is large.
#### clustering key will be itemInSession so that our results are order by it.
#### columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.

In [11]:
create_table_2 = "CREATE TABLE IF NOT EXISTS artist_details"
create_table_2 = create_table_2 + (""" (user_id INT, session_id INT, item_in_session INT, 
                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, 
                                        user_last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session)
                )""")
try:
    session.execute(create_table_2)
except Exception as e:
    print(e)

In [12]:
#insertion for table 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
#select statment for table 2
try:
    rows = session.execute(""" SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_details
                    WHERE user_id = 10
                    AND session_id = 182 
                    """)

except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name,row.song_title,row.user_first_name,row.user_last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### In CQL our query looks like:

#### SELECT firstName, lastName FROM user_details WHERE song = 'All Hands Against His Own'

#### We will name our table users_details
#### Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.
#### The columns of our table will be: song, firstName, lastName and userId.

In [14]:
create_table_3 = "CREATE TABLE IF NOT EXISTS user_details"
create_table_3 = create_table_3 + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, 
                                        PRIMARY KEY ((song_title), user_id)
                )""")
try:
    session.execute(create_table_3)
except Exception as e:
    print(e)
                    

In [15]:
# Insertion for Table 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
# SELECT statement for Table 3
try:
    rows = session.execute(""" SELECT user_first_name, user_last_name FROM user_details WHERE song_title ='All Hands Against His Own'""")
except Exception as e:
    print(e)

for row in rows:
    print(row.user_first_name, row.user_last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions

In [18]:
session.execute("""DROP TABLE songs_details""")

session.execute("""DROP TABLE artist_details""")

session.execute("""DROP TABLE user_details""")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()